## Learning Notes of Renee. Article: 

In [ ]:
# pip install stable-baselines3[extra]

In [ ]:
import gym

from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
import os
import time

In [ ]:
# Saving logs to visulise in Tensorboard, saving models
models_dir = f"models/Mountain-{time.time()}"
logdir = f"logs/Mountain-{time.time()}"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [ ]:
# Parallel environments
env = make_vec_env("MountainCarContinuous-v0", n_envs=1)

# The learning agent and hyperparameters
model = PPO(
    policy=MlpPolicy,
    env=env,
    seed=0,
    batch_size=256,
    ent_coef=0.00429,
    learning_rate=7.77e-05,
    n_epochs=10,
    n_steps=8,
    gae_lambda=0.9,
    gamma=0.9999,
    clip_range=0.1,
    max_grad_norm =5,
    vf_coef=0.19,
    use_sde=True,
    policy_kwargs=dict(log_std_init=-3.29, ortho_init=False),
    verbose=1,
    tensorboard_log=logdir
    )


Using cpu device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:141: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 8`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=8 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [ ]:
#Training and saving models along the way
TIMESTEPS = 20000
for i in range(10): 
    model.learn(total_timesteps=TIMESTEPS,reset_num_timesteps=False, tb_log_name="PPO")
    model.save(f"{models_dir}/{TIMESTEPS*i}")

Streaming output truncated to the last 5000 lines.
|    loss                 | 0.000743     |
|    n_updates            | 247720       |
|    policy_gradient_loss | -0.000501    |
|    std                  | 0.071        |
|    value_loss           | 9.39e-05     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 999          |
|    ep_rew_mean          | -5.51        |
| time/                   |              |
|    fps                  | 149          |
|    iterations           | 2274         |
|    time_elapsed         | 121          |
|    total_timesteps      | 198192       |
| train/                  |              |
|    approx_kl            | 1.937151e-07 |
|    clip_fraction        | 0            |
|    clip_range           | 0.1          |
|    entropy_loss         | 0.441        |
|    explained_variance   | 0.000421     |
|    learning_rate        | 7.77e-05     |
|  

In [ ]:
# Check model performance
# load the best model you observed from tensorboard - the one reach the goal/ obtaining highest return
models_dir = "models/Mountain-1653282767.3143597"
model_path = f"{models_dir}/80000"
best_model = PPO.load(model_path, env=env)

obs = env.reset()
while True:
    action, _states = best_model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    # env.render()  use Python IDE to check, I havn't figure out how to render in Notebook

KeyboardInterrupt: ignored